# 如何处理长文本切分信息丢失的问题 Lost in the middle
头尾识别精度高，中间低。
先切分，再排序，排序将问题相关性高的放在头尾的位置

In [ ]:
! pip3 install sentence-transformers

In [4]:
# from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_ollama import OllamaEmbeddings
# from langchain.document_transformers import LongContextReorder
# from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

# # 使用 huggingface 托管的开源LLM来做嵌入， MiniLM-L6-v2 是一个较小的LLM
# # embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OllamaEmbeddings(
  model="nomic-embed-text:latest",
  base_url="http://localhost:11434"
)

text = [
"清晨的露珠在叶片上闪烁着微弱的光芒。",
"工程师成功研发出新型量子加密通信协议。",
"她每天早晨坚持练习瑜伽以保持身形。",
"远古恐龙化石在博物馆展柜中静默了千万年。",
"程序员熬夜调试代码时发现了一个致命Bug。",
"海边渔民撒下渔网等待着丰收的夜晚来临。",
"天文学家观测到火星表面出现异常陨石坑。",
"小女孩用蜡笔在画布上描绘出奇幻的城堡。",
"实验室里的白鼠在迷宫中找到了隐藏的出口。",
"咖啡馆的钢琴师正在弹奏肖邦的夜曲。",
"滨江大桥的霓虹灯在雨夜映照出绚丽倒影。",
"古董书店老板仔细擦拭着泛黄的线装书。",
"无人机航拍画面显示山火蔓延到了居民区。",
"生物学家在深海热泉口发现了新物种。",
"建筑工地的起重机正将钢材精准吊装到位。",
]

retrieval = Chroma.from_texts(text, embeddings).as_retriever(
  search_kwargs={"k":3}
)

query = "关于宇宙的信息你知道什么？"

# 根据相关性返回文本块
docs = retrieval.get_relevant_documents(query)
docs


[Document(metadata={}, page_content='工程师成功研发出新型量子加密通信协议。'),
 Document(metadata={}, page_content='工程师成功研发出新型量子加密通信协议。'),
 Document(metadata={}, page_content='海边渔民撒下渔网等待着丰收的夜晚来临。')]

以下是为您精心设计的15个短句（均≤18字），包含交叉主题与无关内容混合排列：

人工智能将改变医疗行业（科技）<br>
快递员清晨配送时发现车祸现场（生活）<br>
珊瑚礁因海水酸化大面积白化（自然）<br>
古埃及法老图坦卡蒙墓室新发现壁画（历史）<br>
这家餐厅的招牌菜是麻辣子鸡（美食）<br>
研究表明久坐超过6小时危害健康（健康）<br>
北极科考队成功抵达北极点（探险）<br>
他的小说获本届诺贝尔文学奖提名（文化）<br>
昨夜台风"海葵"袭击东南沿海（气象）<br>
新型纳米材料可降解塑料制品（环保）<br>
患者术后康复需进行物理治疗（医学）<br>
敦煌莫高窟启动数字化保护工程（文物）<br>
咖啡因过量会导致心悸失眠（生活）<br>
太空站宇航员完成植物栽培实验（航天）<br>
市中心广场举办 vintage 时装秀（时尚）<br>

### 检测下这种方案的精度效果

In [ ]:
# from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_ollama import OllamaEmbeddings
# from langchain.document_transformers import LongContextReorder
# from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

# # 使用 huggingface 托管的开源LLM来做嵌入， MiniLM-L6-v2 是一个较小的LLM
# # embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OllamaEmbeddings(
  model="nomic-embed-text:latest",
  base_url="http://localhost:11434"
)

text = [
"人工智能将改变医疗行业",
"快递员清晨配送时发现车祸现场",
"珊瑚礁因海水酸化大面积白化",
"古埃及法老图坦卡蒙墓室新发现壁画",
"这家餐厅的招牌菜是麻辣子鸡",
"研究表明久坐超过6小时危害健康",
"北极科考队成功抵达北极点",
"他的小说获本届诺贝尔文学奖提名",
"昨夜台风'海葵'袭击东南沿海",
"新型纳米材料可降解塑料制品",
"患者术后康复需进行物理治疗",
"敦煌莫高窟启动数字化保护工程",
"咖啡因过量会导致心悸失眠",
"太空站宇航员完成植物栽培实验",
"市中心广场举办丁真珍珠时装秀",
]

# 生成向量数据
retrieval = Chroma.from_texts(text, embeddings).as_retriever(
  search_kwargs={"k":3}
)

query = "关于人工智能的信息你知道什么？"

# 根据相关性返回文本块
docs = retrieval.get_relevant_documents(query)
docs

[Document(metadata={}, page_content='人工智能将改变医疗行业'),
 Document(metadata={}, page_content='工程师成功研发出新型量子加密通信协议。'),
 Document(metadata={}, page_content='工程师成功研发出新型量子加密通信协议。')]